In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import io
#from google.colab import files

import nltk
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('opinion_lexicon')
nltk.download('vader_lexicon')
nltk.download('stopwords')

import string

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import InputExample, InputFeatures

import tensorflow as tf

# Don't worry about the UserWarning about twython (for Twitter)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ll_hayashi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/ll_hayashi/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ll_hayashi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ll_hayashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Create a dataframe from the csv file.
im = pd.read_csv("TA_Training_Set.csv")
# im # Look at it

In [5]:
real = pd.read_csv("TA_Test_Set.csv")

In [6]:
real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Comment  100000 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [7]:
im.columns

Index(['Comment', 'Topic'], dtype='object')

In [8]:
im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Comment  900000 non-null  object
 1   Topic    900000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 13.7+ MB


In [9]:
im["Comment"] = im["Comment"].str.lower()

im.head()

,Comment,Topic
0,very hot and sexy nail color;),21
1,"i know, right? like, an fmea. i definitely kno...",28
2,"hello, we only allow people with an account ag...",5
3,megumin reads das kapital??? new best girl.,2
4,freedom of speech ? but you use a random anon ...,13


In [10]:
real["Comment"] = real["Comment"].str.lower()

real.head()

,Comment
0,i bought a month and a half out on a stock tha...
1,"parity used to be the justification, but that ..."
2,yeah cartel. legolas is gonna shoot your ass d...
3,"i do think he’s ta, but there’s one thing with..."
4,"were trying, let you know if anything works"


# **Setting up the data for binary classification**

In [11]:
# Cleaning [experiment with these choices downstream]
re_tokenizer = RegexpTokenizer("[\\w']+")
im['Comment'] = im['Comment'].astype(str)
# Tokenization using NLTK
im['text_nltk'] = im['Comment'].apply(re_tokenizer.tokenize)
im.head()


,Comment,Topic,text_nltk
0,very hot and sexy nail color;),21,"[very, hot, and, sexy, nail, color]"
1,"i know, right? like, an fmea. i definitely kno...",28,"[i, know, right, like, an, fmea, i, definitely..."
2,"hello, we only allow people with an account ag...",5,"[hello, we, only, allow, people, with, an, acc..."
3,megumin reads das kapital??? new best girl.,2,"[megumin, reads, das, kapital, new, best, girl]"
4,freedom of speech ? but you use a random anon ...,13,"[freedom, of, speech, but, you, use, a, random..."


In [12]:
# Cleaning [experiment with these choices downstream]
re_tokenizer = RegexpTokenizer("[\\w']+")
real['Comment'] = real['Comment'].astype(str)
# Tokenization using NLTK
real['text_nltk'] = real['Comment'].apply(re_tokenizer.tokenize)
real.head()


,Comment,text_nltk
0,i bought a month and a half out on a stock tha...,"[i, bought, a, month, and, a, half, out, on, a..."
1,"parity used to be the justification, but that ...","[parity, used, to, be, the, justification, but..."
2,yeah cartel. legolas is gonna shoot your ass d...,"[yeah, cartel, legolas, is, gonna, shoot, your..."
3,"i do think he’s ta, but there’s one thing with...","[i, do, think, he, s, ta, but, there, s, one, ..."
4,"were trying, let you know if anything works","[were, trying, let, you, know, if, anything, w..."


In [13]:
# Lowercase
im['text_nltk'] = im['text_nltk'].apply(lambda x: [word.lower() for word in x])
im.head(3)

,Comment,Topic,text_nltk
0,very hot and sexy nail color;),21,"[very, hot, and, sexy, nail, color]"
1,"i know, right? like, an fmea. i definitely kno...",28,"[i, know, right, like, an, fmea, i, definitely..."
2,"hello, we only allow people with an account ag...",5,"[hello, we, only, allow, people, with, an, acc..."


In [14]:
# Lowercase
real['text_nltk'] = real['text_nltk'].apply(lambda x: [word.lower() for word in x])
real.head(3)

,Comment,text_nltk
0,i bought a month and a half out on a stock tha...,"[i, bought, a, month, and, a, half, out, on, a..."
1,"parity used to be the justification, but that ...","[parity, used, to, be, the, justification, but..."
2,yeah cartel. legolas is gonna shoot your ass d...,"[yeah, cartel, legolas, is, gonna, shoot, your..."


In [15]:
# Dropping NLTK stop words
stopwords_nltk = set(nltk.corpus.stopwords.words('english'))
im['text_cleaned'] = im['text_nltk'].apply(lambda x: [word for word in x if word not in stopwords_nltk])

# Dropping punctuation and numbers:
punc = string.punctuation + '--' # Add characters as necessary 
im['text_cleaned'] = im['text_cleaned'].apply(lambda x: [word for word in x if word not in punc])

digits = string.digits 
im['text_cleaned'] = im['text_cleaned'].apply(lambda x: [word for word in x if word not in digits])
im['text_cleaned'] = im['text_cleaned'].apply(lambda x: [word for word in x if len(word)>1]) 

# from my understanding, words like "https", "'s" and "'" can also be dropped
additional = ["https", "'s", "’","'re", "amp", ".co", "co", "“", "”", "''", "``", "weâ€™re", "//t.co/"]
im['text_cleaned'] = im['text_cleaned'].apply(lambda x: [word for word in x if word not in additional])
im['text_cleaned_string'] = [' '.join(map(str, l)) for l in im['text_cleaned']]

im['text_cleaned_string'] = [t.replace("//t.co/", "") for t in im['text_cleaned_string']]

im.head(3)

,Comment,Topic,text_nltk,text_cleaned,text_cleaned_string
0,very hot and sexy nail color;),21,"[very, hot, and, sexy, nail, color]","[hot, sexy, nail, color]",hot sexy nail color
1,"i know, right? like, an fmea. i definitely kno...",28,"[i, know, right, like, an, fmea, i, definitely...","[know, right, like, fmea, definitely, know, on...",know right like fmea definitely know one want ...
2,"hello, we only allow people with an account ag...",5,"[hello, we, only, allow, people, with, an, acc...","[hello, allow, people, account, age, days, 50,...",hello allow people account age days 50 comment...


In [16]:
# Dropping NLTK stop words
stopwords_nltk = set(nltk.corpus.stopwords.words('english'))
real['text_cleaned'] = real['text_nltk'].apply(lambda x: [word for word in x if word not in stopwords_nltk])

# Dropping punctuation and numbers:
punc = string.punctuation + '--' # Add characters as necessary 
real['text_cleaned'] = real['text_cleaned'].apply(lambda x: [word for word in x if word not in punc])

digits = string.digits 
real['text_cleaned'] = real['text_cleaned'].apply(lambda x: [word for word in x if word not in digits])
real['text_cleaned'] = real['text_cleaned'].apply(lambda x: [word for word in x if len(word)>1]) 

# from my understanding, words like "https", "'s" and "'" can also be dropped
additional = ["https", "'s", "’","'re", "amp", ".co", "co", "“", "”", "''", "``", "weâ€™re", "//t.co/"]
real['text_cleaned'] = real['text_cleaned'].apply(lambda x: [word for word in x if word not in additional])
real['text_cleaned_string'] = [' '.join(map(str, l)) for l in real['text_cleaned']]

real['text_cleaned_string'] = [t.replace("//t.co/", "") for t in real['text_cleaned_string']]

real.head(3)

,Comment,text_nltk,text_cleaned,text_cleaned_string
0,i bought a month and a half out on a stock tha...,"[i, bought, a, month, and, a, half, out, on, a...","[bought, month, half, stock, almost, option, v...",bought month half stock almost option volume l...
1,"parity used to be the justification, but that ...","[parity, used, to, be, the, justification, but...","[parity, used, justification, days, free, agen...",parity used justification days free agency way...
2,yeah cartel. legolas is gonna shoot your ass d...,"[yeah, cartel, legolas, is, gonna, shoot, your...","[yeah, cartel, legolas, gonna, shoot, ass]",yeah cartel legolas gonna shoot ass


In [17]:
pd.set_option('display.max_rows', None)

In [18]:
# Additional cleaning:
from collections import Counter, defaultdict
# Count and list most frequent tokens
counter = Counter()
im['text_cleaned'].map(counter.update)
most_freq_train = pd.DataFrame(counter.most_common()) # Or add number within parens to list only x most common
most_freq_train.columns = ['token', 'count']



In [19]:
most_freq_train.head(50)

,token,count
0,like,128616
1,people,82484
2,would,80870
3,one,76515
4,get,72486
5,think,63561
6,com,59970
7,time,58380
8,reddit,52472
9,i'm,52174


In [20]:
additional = ["www", "com","gt","i'm’", "he's","that's", "us","still","could"]
im['text_cleaned'] = im['text_cleaned'].apply(lambda x: [word for word in x if word not in additional])
im['text_cleaned_string'] = [' '.join(map(str, l)) for l in im['text_cleaned']]

im['text_cleaned_string'] = [t.replace("//t.co/", "") for t in im['text_cleaned_string']]

im.head(3)

,Comment,Topic,text_nltk,text_cleaned,text_cleaned_string
0,very hot and sexy nail color;),21,"[very, hot, and, sexy, nail, color]","[hot, sexy, nail, color]",hot sexy nail color
1,"i know, right? like, an fmea. i definitely kno...",28,"[i, know, right, like, an, fmea, i, definitely...","[know, right, like, fmea, definitely, know, on...",know right like fmea definitely know one want ...
2,"hello, we only allow people with an account ag...",5,"[hello, we, only, allow, people, with, an, acc...","[hello, allow, people, account, age, days, 50,...",hello allow people account age days 50 comment...


In [21]:
# Additional cleaning:
from collections import Counter, defaultdict
# Count and list most frequent tokens
counter = Counter()
real['text_cleaned'].map(counter.update)
most_freq_test = pd.DataFrame(counter.most_common()) # Or add number within parens to list only x most common
most_freq_test.columns = ['token', 'count']

In [22]:
most_freq_test.head(50)

,token,count
0,like,14334
1,people,9087
2,would,9004
3,one,8713
4,get,7992
5,think,7121
6,com,6894
7,time,6597
8,reddit,5975
9,know,5798


In [23]:
additional = ["www", "com","gt","i'm’", "he's","that's", "us","still","could"]
real['text_cleaned'] = real['text_cleaned'].apply(lambda x: [word for word in x if word not in additional])
real['text_cleaned_string'] = [' '.join(map(str, l)) for l in real['text_cleaned']]

real['text_cleaned_string'] = [t.replace("//t.co/", "") for t in real['text_cleaned_string']]

real.head(3)

,Comment,text_nltk,text_cleaned,text_cleaned_string
0,i bought a month and a half out on a stock tha...,"[i, bought, a, month, and, a, half, out, on, a...","[bought, month, half, stock, almost, option, v...",bought month half stock almost option volume l...
1,"parity used to be the justification, but that ...","[parity, used, to, be, the, justification, but...","[parity, used, justification, days, free, agen...",parity used justification days free agency way...
2,yeah cartel. legolas is gonna shoot your ass d...,"[yeah, cartel, legolas, is, gonna, shoot, your...","[yeah, cartel, legolas, gonna, shoot, ass]",yeah cartel legolas gonna shoot ass


In [24]:
# Look at class balance/imbalance
im['Topic'].value_counts()

20    22572
28    22570
6     22564
15    22550
18    22548
1     22539
33    22533
25    22533
10    22530
29    22527
24    22523
11    22521
27    22519
12    22515
3     22513
19    22513
30    22510
32    22503
4     22503
17    22497
7     22492
8     22492
21    22491
38    22491
22    22486
36    22486
31    22483
35    22483
39    22482
9     22482
14    22469
2     22468
37    22464
23    22460
34    22458
16    22453
26    22452
13    22450
5     22439
40    22436
Name: Topic, dtype: int64

## ****

In [25]:
#X_train, X_test, Y_train, Y_test = train_test_split(im['text_cleaned'], im['Topic'],)

In [26]:
X_train = im['text_cleaned']
Y_train =im['Topic']
X_test = real['text_cleaned']

In [27]:
# Convert each X list item to strings for the next step
X_train_str = [' '.join(map(str, l)) for l in X_train]
X_test_str = [' '.join(map(str, l)) for l in X_test]

In [28]:
# Create TF-IDF vectors
vectorizer = TfidfVectorizer(ngram_range=(1,2)) 
X_train_tfidf = vectorizer.fit_transform(X_train_str)
X_test_tfidf = vectorizer.transform(X_test_str)

In [29]:
# Look at this object -- what does the note mean?
X_train_tfidf

<900000x5622053 sparse matrix of type '<class 'numpy.float64'>'
	with 24719630 stored elements in Compressed Sparse Row format>

# ** SVM**

In [30]:

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, roc_auc_score, average_precision_score 
from sklearn.metrics import auc, roc_curve, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import InputExample, InputFeatures
# from sentence_transformers import SentenceTransformer


In [ ]:
# Train the SVM model
svm_model = LinearSVC(random_state = 72)
#or
#svm_model = LinearSVC()
svm_model.fit(X_train_tfidf, Y_train)

In [ ]:
# Generate predictions on the test set
Y_pred_svm = svm_model.predict(X_test_tfidf)
Y_pred_svm

In [ ]:
# Generate confusion matrix
#cm = confusion_matrix(Y_test, Y_pred_svm, labels=svm_model.classes_)
#cm_display = ConfusionMatrixDisplay(cm, display_labels=svm_model.classes_).plot()


In [ ]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
svc = LinearSVC(random_state = 1, max_iter = 10000, C = 0.5)
svc.fit(X_train_tfidf, Y_train)
print("train score:",svc.score(X_train_tfidf, Y_train))
#print("test score:",svc.score(X_test_tfidf, Y_test))

In [ ]:
predicted = svc.predict(X_test_tfidf)

In [ ]:
predicted

In [ ]:
Topic = pd.DataFrame(predicted)

In [ ]:
Comment = pd.DataFrame(real)
Comment.head()

In [ ]:
Topic.head()

In [ ]:
Topic.rename( columns={0:'Topic'}, inplace=True )

In [ ]:
Topic.columns

In [ ]:
Comment['Topic']= Topic

In [ ]:
Comment.head()

In [ ]:
#final = pd.DataFrame({"Comment": ["Comment", 'text_nltk','text_cleaned','text_cleaned_string'],
#                    "Topic": ['Topic']})

In [ ]:
final

In [48]:
pd.DataFrame(predicted).to_csv("T_lin.csv")